<a href="https://colab.research.google.com/github/stephenlovoi/twitter-sentiment-analysis/blob/main/trump_rds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# OPEN FILE IN GOOGLE COLAB

import os

spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-06-19 14:02:38--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.33MB/s    in 0.2s    

2021-06-19 14:02:39 (4.33 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://trump-twitter-bucket.s3.us-east-2.amazonaws.com/trump_archive.csv"
spark.sparkContext.addFile(url)

trump_archive_df = spark.read.csv(SparkFiles.get("trump_archive.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
trump_archive_df.show()

+---+--------------------+---------+---------+--------+-------------------+
|_c0|                text|isRetweet|favorites|retweets|               date|
+---+--------------------+---------+---------+--------+-------------------+
|  0|Republicans and D...|        f|       49|     255|2011-08-02 18:07:48|
|  1|The Unsolicited M...|        f|    80527|   23502|2020-09-12 20:10:58|
|  2|The threshold ide...|        f|   493076|  100609|2020-11-09 19:36:26|
|  3|Nevada is turning...|        f|   363489|   78378|2020-11-09 19:54:33|
|  4|Wisconsin is look...|        f|   347994|   61006|2020-11-09 20:02:19|
|  5|Pennsylvania prev...|        f|   479292|   81458|2020-11-09 20:17:47|
|  6|Just landed in Oh...|        f|   139153|   14000|2020-10-24 20:13:08|
|  7|Georgia will be a...|        f|   637719|   94570|2020-11-09 20:21:43|
|  8|How can you certi...|        f|   211508|   56945|2021-01-04 15:07:16|
|  9|Polls numbers are...|        f|   280261|   52623|2020-10-16 11:24:43|
| 10|Heading

In [6]:
url="https://trump-twitter-bucket.s3.us-east-2.amazonaws.com/testquotes.csv"
spark.sparkContext.addFile(url)

testquote_df = spark.read.csv(SparkFiles.get("testquotes.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
testquote_df.show()

+---+--------------------+
|_c0|               quote|
+---+--------------------+
|  0| “I am the only o...|
|  1| “I deal with for...|
|  2| “With the coldes...|
|  3| “I don't like to...|
|  4| “This very expen...|
|  5| “A lot of people...|
|  6| “You have to thi...|
|  7| “Somebody's got ...|
|  8| “Sometimes by lo...|
|  9| “Life is actuall...|
| 10| “Nothing is easy...|
| 11| “I don't think I...|
| 12| “Sheer persisten...|
| 13| “My father built...|
| 14| “You can't be sc...|
| 15| “There's always ...|
| 16| “Well, I am a Re...|
| 17| “I fire people t...|
| 18| “I like thinking...|
| 19| “Sometimes I don...|
+---+--------------------+
only showing top 20 rows



In [7]:
trump_archive_df.describe()

DataFrame[summary: string, _c0: string, text: string, isRetweet: string, favorites: string, retweets: string, date: string]

In [8]:
clean_trump_archive_df = trump_archive_df.select(["text", "favorites", "retweets"])

In [13]:
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import IntegerType, StructField, StructType, StringType


clean_trump_archive_df = clean_trump_archive_df.withColumn("favorites", clean_trump_archive_df["favorites"].cast(IntegerType()))
clean_trump_archive_df = clean_trump_archive_df.withColumn("retweets", clean_trump_archive_df.retweets.cast(IntegerType()))

testquote_df = testquote_df.withColumn("_c0", testquote_df["_c0"].cast(IntegerType()))

[('_c0', 'int'), ('quote', 'string')]

In [14]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://trump-twitter-db.cyjaclylpg2n.us-east-2.rds.amazonaws.com:5432/trump_archive_db"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write data into SQL
clean_trump_archive_df.write.jdbc(url=jdbc_url, table='trump_tweets', mode=mode, properties=config)

In [15]:
testquote_df.write.jdbc(url=jdbc_url, table='trump_quotes', mode=mode, properties=config)